In [10]:
from pymongo import MongoClient

# Connect to the server with the atlas cnx string colab can't connect to localhost
cluster = MongoClient('mongodb://localhost:27017/')

# Connect to the Database where the images will be stored.
database = cluster['Classif_urban_DB']
collection =  database['test']
# post has strct of json data & always start with _id generated automatically if not specific
post = {
    "id_" : 0,
    "test": "first insert into MongoDB"
}
collection.insert_one(post)


ServerSelectionTimeoutError: ignored

In [3]:
"""import gridfs

#Create an object of GridFs for the above database.
fs = gridfs.GridFS(database)

#define an image object with the location.
file = "/content/image_part_001.jpg"

#Open the image in read-only format.
with open(file, 'rb') as f:
    contents = f.read()

#Now store/put the image via GridFs object.
fs.put(contents, filename="file")"""

ServerSelectionTimeoutError: ignored